In [1]:
# Importar librerías necesarias
import numpy as np
import sympy as sp

## Comprobación por Roché-Frobenius
    - Si Rank(A) <> Rank(A*) --> Incompatible
    - Si Rank(A*) == nº incognitas --> Compatible determinado
    - Si Rank(A*) < nº incognitas --> Compatible indeterminado

In [115]:
def check_rank(A, E):
    extended_rank = np.linalg.matrix_rank(E)
    if (np.linalg.matrix_rank(A) != extended_rank):
        raise ValueError("La matriz introducida es incompatible")
    if (extended_rank < A.shape[1]):
        raise ValueError("La matriz introducida es compatible indeterminado")

# **Práctica 6:** Resolución de sistemas de ecuaciones lineales con métodos iterativos (Jacobi y Gauss-Seidel)

El objetivo de esta práctica es programar los métodos de resolución iterativa de sistemas de ecuaciones lineales explicados en clase. 

Los programas deben recibir la matriz extendida del sistema $[A, b]$ (matriz de coeficientes y la última columna, la matriz de términos independientes).

**NOTAS:** 
* Los métodos a implementar en las secciones 1.1 y 1.2 deben lanzar una Excepción si se recibe como entrada un sistema incompatible o compatible indeterminado.
* Considerar un criterio de parada estudiado en la clase teórica. Considerar el error de $0.001$.
* Tener en cuenta que el método pueda no converger. Por tanto, establecer un
número máximo de iteraciones, por ejemplo $maxIter = 1000$
* Al final, el programa deberá dar un aviso en pantalla informando si ha encontrado la solución o si la parada fue por la no convergencia. 

# 1. Implementación de los métodos

## 1.1. Método de Jacobi

Implementa el método de Jacobi para resolver sistemas de ecuaciones lineales explicado en teoría.

In [136]:
def Jacobi(M):
    # Comprueba que sea compatible determinado
    size = M.shape[1] - 1
    try:
        check_rank(M[:,range(size)], M)
    except ValueError as exception:
        print(exception)
        return None
    
    # Comienza el algoritmo
    diagonal_mat = np.diag(np.diag(M))
    inv_diag_mat = (-1) * np.linalg.inv(diagonal_mat)
    cleared_mat = np.subtract(M[:, range(size)], diagonal_mat)
    
    iters = 0
    tol = 0.001
    maxIter = 1000
    sol_ant = np.ones(M.shape[0])
    sol = np.add(np.matmul(inv_diag_mat, np.matmul(cleared_mat, sol_ant)), np.matmul(inv_diag_mat, M[:, size]))
    
    # x = -D^-1 * A_2 * x + D^-1 * b
    while True:
        
        if (iters >= maxIter):
            print("Ejecucion terminada por iteraciones superadas")
            break
        
        op1 = np.matmul(inv_diag_mat, M[:, size])
        op2 = np.matmul(cleared_mat, sol_ant) ## Invalid value in matmul, last columns have NaN

        op3 = np.matmul(inv_diag_mat, op2)
        sol = np.add(op3, op1)
        
        print(np.linalg.norm(sol- sol_ant)) ## Mirar como tratar tolerancia
        
        if (np.linalg.norm(sol_ant - sol) < tol):
            break
        
        sol_ant = sol
        iters += 1
        
    return sol

## 1.2. Método de Gauss-Seidel

Implementa el método de Gauss-Seidel para resolver sistemas de ecuaciones lineales.

In [ ]:
def GaussSeidel(M):
    # COMPLETE
    
    pass

# 2. Aplicación de los métodos

En este apartado, utiliza uno de los dos métodos implementados en la sección anterior para obtener sus resultados.

NOTA: Intentar reescribir el sistema para garantizar la convergencia, si posible.


## 2.1. Ejercicio A

Resuelve los siguientes sistemas:

1.  $\begin{cases}
    2x_1 + 4x_3 = 0\\
    6x_1 - 3x_2 + 7x_3 = 1\\
    -4x_1 + 6x_2 + 2x_3 = 3
    \end{cases}$
    
3. $\begin{cases}
    6x_1 - 9x_2 + x_3 + 4x_5 = -5\\
    -2x_1 + 3x_2 - x_3 + 5x_4 + 9x_5 = -24\\
    x_2 - x_3 + 7x_4 + 2x_5 = -10\\
    5x_1 - 6x_2 + 8x_3 - x_4 = 5\\
    3x_1 + 7x_2 - 2x_3 + x_4 + 5x_5 = 2
    \end{cases}$

Devuelve un ndarray para cada sistema


In [137]:
sys1_coef = np.array([[2, 0, 4], [6, -3, 7], [-4, 6, 2]])
sys1_ind = np.array([[0], [1], [3]])
sys1_ext = np.hstack((sys1_coef, sys1_ind))
#res_1 = Jacobi(sys1_ext)

sys2_coef = np.array([[6, -9, 1, 0, 4], [-2, 3, -1, 5, 9], [0, 1, 1, 7, 2], [5, -6, 8, -1, 0], [3, 7, -2, 1, 5]])
sys2_ind = np.array([[-5], [-24], [-10], [5], [2]])
sys2_ext = np.hstack((sys2_coef, sys2_ind))
res_2 = Jacobi(sys2_ext)

#print(f'Solucion 1: {res_1}')
print(f'Solucion 2: {res_2}')

11.983368103797494
79.39271587074806
543.9516072066668
3885.1051237081033
25019.90628543056
177030.5743694331
1161979.4718438364
8159319.586655516
53890492.40133403
375408286.0842579
2500659943.2988796
17271911762.19288
116045750863.43326
794503319393.0444
5385517311600.075
36544038036827.94
249925089605610.2
1680873976763303.0
1.159694162280172e+16
7.731915466159542e+16
5.38017135701718e+17
3.557196713057814e+18
2.4953869070161502e+19
1.6369396442873882e+20
1.157021552207758e+21
7.535211015552545e+21
5.362684145585287e+22
3.4699890699021306e+23
2.4844981464464033e+24
1.5986738546777645e+25
1.1505155882275209e+26
7.369130171814826e+26
5.325111699940916e+27
3.398742935419982e+28
2.4634046279568205e+29
1.56849188553292e+30
1.1389522319312388e+31
7.24300398604192e+31
5.2630005816055236e+32
3.3468205641383333e+33
2.4306276652226186e+34
1.5474696802556505e+35
1.1219230342793724e+36
7.159466711094798e+36
5.175750455298031e+37
3.314324754979371e+38
2.386485276026767e+39
1.5351366464309874e+40

/tmp/ipykernel_4346/2105804222.py:29: RuntimeWarning: overflow encountered in matmul
  op2 = np.matmul(cleared_mat, sol_ant) ## Invalid value in matmul, last columns have NaN
/tmp/ipykernel_4346/2105804222.py:29: RuntimeWarning: invalid value encountered in matmul
  op2 = np.matmul(cleared_mat, sol_ant) ## Invalid value in matmul, last columns have NaN
/tmp/ipykernel_4346/2105804222.py:31: RuntimeWarning: invalid value encountered in matmul
  op3 = np.matmul(inv_diag_mat, op2)


## 2.2. Ejercicio B

Problema. Un bar va a renovar el mobiliario y te ha pedido asesoramiento. Desean instalar mesas de tres tamaños: 
* Pequeñas, de 5 asientos cada una
* Medianas premium, de 10 asientos cada una
* Grandes, de 20 asientos cada una

Desean que en total haya 50 mesas, con capacidad para 525 comensales. Se ha calculado que el coste de cada mesa pequeña (con sus asientos) es de 100 euros; la mediana (con asientos), más cara, 500 euros; y la grande (con asientos) 250 euros. Se tiene un presupuesto de 17550 euros.

Resolver: 

1. Plantear el sistema de ecuaciones a resolver
2. Utilizar el método de Jacobi o Gauss-Seidel para calcular cuántas mesas deberán encargarse. Prueba a inicializar la solución con $\mathbf{x}^{(0)} = (0,0,0)$ y luego con otra que satisfaga la primera ecuación como por ejemplo $\mathbf{x}^{(0)} = (10,30,10)$

1. Sistema de ecuaciones. Planteamiento:

2. Resolución. Devuelve un ndarray para la solución del sistema.

## 2.3. Ejercicio C

Estudia la solución del sistema formado por 
$$ 4x_1 + x_2 = 1 $$
$$ 4x_2 + x_3 = 2 $$
$$ 4x_3 + x_4 = 3 $$
$$ ... $$
$$ 4x_{10} + x_{11} = 10 $$
$$ 4x_{11} = 11 $$


1. Representar las matrices $A$ y $b$ del sistema

2. Obtener la solución de las variables $x_1, x_2, \ldots, x_{11}$. Devuelve un ndarray de 11 posiciones para la solución del sistema.